In [18]:
import tensorflow as tf

In [19]:

base_dir = 'dataset/train'

IMAGE_SIZE = 224 
BATCH_SIZE = 32 
epochs = 50
class_number = 3
INIT_LR = 1e-6

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale = 1./255,
    validation_split=0.2,   
)

train_generator = datagen.flow_from_directory( 
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  
    batch_size = BATCH_SIZE, 
    subset='training' 
)

val_generator = datagen.flow_from_directory(  
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

print(train_generator.class_indices) 
labels = '\n'.join(sorted(train_generator.class_indices.keys())) 

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 
base_model = tf.keras.applications.MobileNetV2( 
    input_shape=IMG_SHAPE,
    include_top=False, 
    weights='imagenet',   
)

base_model.trainable=False 
model = tf.keras.Sequential([ 
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'), 
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.GlobalAveragePooling2D(), 
  tf.keras.layers.Dense(class_number,activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=INIT_LR), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)


In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss' ,patience=2, mode='min',verbose=1),
    tf.keras.callbacks.ModelCheckpoint(filepath='models/2haziran_Mobilenetv2_model.{epoch:02d}-{val_loss:.5f}.h5',monitor='val_loss',mode='min',save_best_only=True),
]

history = model.fit(
    train_generator, 
    epochs = epochs, 
    callbacks=[my_callbacks],
    validation_data=val_generator,
)

model.summary()


In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir) 

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert() 

In [23]:
with open('2haziran_mobilenetv2.tflite', 'wb') as f: 
  f.write(tflite_model)

In [24]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, epochs), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, epochs), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("1haziran_mobilenetv2")